In [43]:
import os
import shutil
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import fiona
import logging
from pathlib import Path

In [28]:
def table_to_shp(input_file, output_file, lon_col, lat_col, input_crs=4326, output_crs=4545):
    # 设置fiona支持utf-8编码
    fiona.supported_drivers['ESRI Shapefile'] = 'rw'
    # 读取输入文件，如果是dataframe，则直接转换为shapefile
    if isinstance(input_file, pd.DataFrame):
        df = input_file
        # 创建几何列
        geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
        
        # 创建GeoDataFrame
        gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=f"EPSG:{input_crs}")
    else:
        _, file_extension = os.path.splitext(input_file)
        if file_extension.lower() == '.xlsx':
            df = pd.read_excel(input_file)
        elif file_extension.lower() == '.csv':
            df = pd.read_csv(input_file, encoding='utf-8')
        else:
            raise ValueError("不支持的文件格式。请使用.xlsx或.csv文件。")
        # 创建几何列
        geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
        
        # 创建GeoDataFrame
        gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=f"EPSG:{input_crs}")
    
    # 如果需要,转换坐标系
    if input_crs != output_crs:
        gdf = gdf.to_crs(epsg=output_crs)
    
    # 保存为shapefile,使用utf-8编码
    gdf.to_file(output_file, driver="ESRI Shapefile", encoding='utf-8')
    
    print(f"Shapefile已保存至: {output_file}")

# 提取土壤质地数据

In [29]:
# 准备土壤质地数据
trzd_path = r"C:\Users\Runker\Desktop\mdb\result\物理性状.xlsx"
# 读取DWJD、DWWD、TRZD列
trzd_df = pd.read_excel(trzd_path,usecols=["DWJD","DWWD","TRZD"])
# 去除空值
trzd_df = trzd_df.dropna()
# 去除'/'值
trzd_df = trzd_df[trzd_df["TRZD"] != "/"]
# 去除重复值
trzd_df = trzd_df.drop_duplicates(subset=["DWJD","DWWD"])
# 重置索引
trzd_df = trzd_df.reset_index(drop=True)
# 映射字典
trzd_dict = {'壤土':1,'粉(砂)质壤土':2,'砂质壤土':3,'粉(砂)质黏壤土':4,'黏壤土':5,'砂质黏壤土':6,'壤质黏土':7,'黏土':8,'砂质黏土':9,'重黏土':10,'粉(砂)质黏土':11,'砂土及壤质砂土':12}
# 将TRZD列映射为数字
trzd_df["TRZD_CODE"] = trzd_df["TRZD"].map(trzd_dict)
# 将TRZD列转换为整数
trzd_df["TRZD_CODE"] = trzd_df["TRZD_CODE"].astype(int)
print(trzd_df["TRZD_CODE"].value_counts().to_list(),len(trzd_df["TRZD_CODE"].value_counts().to_list()))
# 保存为shp
table_to_shp(trzd_df,r"C:\Users\Runker\Desktop\genarate_feature\shp_file\土壤质地.shp","DWJD","DWWD",input_crs=4326,output_crs=4545)


# 提取河流数据

In [ ]:
# 读取三调数据
sd_path = r"F:\cache_data\shp_file\sb\sb_sd_dltb.shp"
# 读取河流数据
sd_df = gpd.read_file(sd_path,encoding="utf-8")
# 提取DLMC为 "河流水面": "1101", "湖泊水面": "1102", "水库水面": "1103", "坑塘水面": "1104", "沟渠": "1107",
river_df = sd_df[sd_df["DLMC"].isin(["河流水面", "湖泊水面", "水库水面", "坑塘水面", "沟渠"])]
# 删除多余列，仅保留DLMC、DLMC_CODE、DLMC_NAME、geometry
river_df = river_df[["DLMC","DLBM", "geometry"]]
# 保存为shp
river_df.to_file(r"C:\Users\Runker\Desktop\genarate_feature\shp_file\河流.shp",driver="ESRI Shapefile",encoding='utf-8')


# 复制DEM,PRE,TWI,坡位数据

In [46]:

origin_dem_path = r"F:\tif_features\county_feature\sb\dem.tif"
origin_pre_path = r"F:\tif_features\county_feature\sb\pre2022mean.tif"
origin_twi_path = r"F:\tif_features\county_feature\sb\topographicwetnessindex.tif"
origin_slope_position_path = r"F:\tif_features\county_feature\sb\slopeclasssmooth.tif"

# 复制DEM,PRE,地质数据
dem_output_path = r"C:\Users\Runker\Desktop\genarate_feature\raster_file\dem.tif"
pre_output_path = r"C:\Users\Runker\Desktop\genarate_feature\raster_file\pre2022mean.tif"
twi_output_path = r"C:\Users\Runker\Desktop\genarate_feature\raster_file\twi.tif"
slope_position_output_path = r"C:\Users\Runker\Desktop\genarate_feature\raster_file\slope_position.tif"
# 复制DEM,PRE,地质数据
shutil.copy(origin_dem_path, dem_output_path)
shutil.copy(origin_pre_path, pre_output_path)
shutil.copy(origin_twi_path, twi_output_path)
shutil.copy(origin_slope_position_path, slope_position_output_path)


'C:\\Users\\Runker\\Desktop\\genarate_feature\\raster_file\\slope_position.tif'